In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
    

%matplotlib inline

In [2]:
movie_budgets = 'tn.movie_budgets.csv.gz'

df_movie_budgets = pd.read_csv(f'zippedData/{movie_budgets}')
df_movie_budgets
df_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [3]:
def reformat_money(money):
    money = money.replace(',','')
    return int(money[1:])

df_movie_budgets['production_budget'] = df_movie_budgets['production_budget'].apply(reformat_money)
df_movie_budgets['domestic_gross'] = df_movie_budgets['domestic_gross'].apply(reformat_money)
df_movie_budgets['worldwide_gross'] = df_movie_budgets['worldwide_gross'].apply(reformat_money)

df_movie_budgets

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0
5778,79,"Apr 2, 1999",Following,6000,48482,240495
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0


In [4]:
# Create revenue and profit column
revenue = df_movie_budgets['domestic_gross'] + df_movie_budgets['worldwide_gross']
profit = revenue - df_movie_budgets['production_budget']

df_movie_budgets['revenue'] = revenue
df_movie_budgets['profit'] = profit

# df_movie_budgets['revenue'] = revenue
df_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,revenue,profit
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,3536852904,3111852904
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,1286727750,876127750
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,192524700,-157475300
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,1862019831,1531419831
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,1936903129,1619903129


In [5]:
movie_release = df_movie_budgets.copy() #we're going to need a copy of this later
movie_release['month'] = movie_release['release_date'].str[:3] #change these into strings
movie_release['year'] = movie_release['release_date'].str[7:].astype(int)
movie_release = movie_release.drop_duplicates().dropna().loc[(movie_release!=0).all(1)] # goodbye Nans, zeros, and duplicates

movie_release

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,revenue,profit,month,year
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,3536852904,3111852904,Dec,2009
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,1286727750,876127750,May,2011
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,192524700,-157475300,Jun,2019
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,1862019831,1531419831,May,2015
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,1936903129,1619903129,Dec,2017
...,...,...,...,...,...,...,...,...,...,...
5775,76,"May 26, 2006",Cavite,7000,70071,71644,141715,134715,May,2006
5776,77,"Dec 31, 2004",The Mongol King,7000,900,900,1800,-5200,Dec,2004
5778,79,"Apr 2, 1999",Following,6000,48482,240495,288977,282977,Apr,1999
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338,2676,-2324,Jul,2005


In [8]:
movie_release.loc[movie_release.duplicated(keep=False),:]


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,revenue,profit,month,year


In [ ]:
1rearrange_columns = list(movie_release.columns)
rearrange_columns[0],rearrange_columns[8] = rearrange_columns[8],rearrange_columns[0]
drearrange_columns[2],rearrange_columns[9] = rearrange_columns[9],rearrange_columns[2]
rearrange_columns

In [ ]:
movie_release = movie_release[rearrange_columns]
movie_release

In [ ]:
movie_release_years = movie_release['year'].value_counts(ascending = False) # frequency
# movie_release_years = movie_release['year'].sort_values(ascending=False) # min year 1915 max year 2020
movie_release_years

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.bar(movie_release_years.index,movie_release_years, color='Orange')
ax.set_xlabel('Months');
ax.set_ylabel('Frequency of Movies')
ax.set_title('Frequency of Movies Released by Years')

Lets look at movies that were released from 2009 to 2019. Not many movies were released in 2020 because of COVID.

In [ ]:
movie_release_10years = movie_release[movie_release['year'] >= 2009]
movie_release_10years

In [ ]:
# create a dataframe from month and profit
profit_by_month = movie_release_10years[['month','profit']]
profit_by_month = profit_by_month.sort_values('profit',ascending=False)

In [ ]:
# plot it like its hotttt
fig, ax = plt.subplots(figsize=(15,6))
ax.bar(profit_by_month['month'],
       profit_by_month['profit'], 
       color=(profit_by_month['profit']>0).map({True:'Green',False:'Red'})
      )
ax.set_xlabel('Months')
ax.set_ylabel('Profit (in Billions)')
ax.set_title('Profit from 2009-2019 by Month')

Overall the months that made the most profit from the years 2009-2019 is December, April, and June. Although June lost a significant amount.

In [ ]:
# Moving on lets load IMDB
imbd_title = 'imdb.title.basics.csv.gz'

imbd_title_data = pd.read_csv(f'zippedData/{imbd_title}')
imbd_title_data

In [ ]:
# check for no null-values
imbd_title_data.info()
imbd_title_data.isna().sum()

In [ ]:
imbd_title_data = imbd_title_data.drop(["original_title", "runtime_minutes", "tconst"], axis = 1) # drop the missing values
imbd_title_data.dropna(inplace=True)  # drop missing value genres
imbd_title_data = imbd_title_data.rename(columns={"primary_title": "movie"}) # rename column for later merge
imbd_title_data.isna().sum()
imbd_title_data.info() 

In [ ]:
# Hello new clean data
imbd_title_data

In [ ]:
# lets use the first genre listed in the string of genres to keep it consistent
def first_genre_listed(n):
    # Return only the first genre listed
    n = str(n)
    if n == '\\N':
        return ''
    return n.split(',')[0].strip()

imbd_title_data['genres'] = imbd_title_data['genres'].apply(first_genre_listed)

imbd_title_data

In [ ]:
#lets stick to 10 years we got something good going on
imbd_movie_budget = imbd_title_data.merge(movie_release_10years, on = "movie", how="inner")
imbd_movie_budget

In [ ]:
imbd_movie_budget = imbd_movie_budget.drop(["start_year"], axis = 1) # remove the columns we don't need
imbd_movie_budget[imbd_movie_budget.duplicated(subset=['movie'])] # check for duplicated movie titles
imbd_movie_budget = imbd_movie_budget.drop_duplicates(subset=['movie']) # goodbye
imbd_movie_budget

In [ ]:
# lets find the goat genres by profit
# On average which genre makes the most profiit?
genres_and_profit_avg = imbd_movie_budget.groupby(['genres']).profit.mean().sort_values(ascending=False)
genres_and_profit_avg[:14]

In [ ]:
sns.color_palette("crest", as_cmap=True)
fig, ax = plt.subplots(figsize=(20,8))
data = genres_and_profit_avg[:14]
genre = data.index 
total = data.values
# create bar chart 
sns.barplot(genre, total) 
# set title and labels 
ax.set_title('Average Genre Profit of 2009-2019') 
ax.set_ylabel('Profit Average (In Billions)')
plt.rc('xtick', labelsize=16);

On average the top profiting genres are Animation, Musical, Adventure, and Action. Lets compare it to the median.

In [ ]:
genres_and_profit_med = imbd_movie_budget.groupby(['genres']).profit.median().sort_values(ascending=False)
genres_and_profit_med[:14]

In [ ]:
sns.color_palette("crest", as_cmap=True)
fig, ax = plt.subplots(figsize=(20,8))
data = genres_and_profit_med[:14]
genre = data.index 
total = data.values
# create bar chart 
sns.barplot(genre, total) 
# set title and labels 
ax.set_title('Overall Median Genre Profit of 2009-2019') 
ax.set_ylabel('Profit Median (In Billions)')
plt.rc('xtick', labelsize=16);

The Median of the genres are fairy close to each other meaning that the dataset is normally distributed and not skewed so we can rely on the mean to make our reccomendations. We still stick with recommending Animation, Musical, Adventure, Action, and Mystery as the top 5 genres to reccomend.